In [16]:
import sqlite3
import pandas as pd

In [17]:
conn = sqlite3.connect("sqlite.db")
curs = conn.cursor()

In [18]:
df = pd.read_sql("""
                SELECT *
                FROM  Orders
                 """, conn)
df

,OrderID,CustomerID,EmployeeID,OrderDate,ShipperID
0,10248,90,5,2023-07-04,3
1,10249,81,6,2023-07-05,1
2,10250,34,4,2023-07-08,2
3,10251,84,3,2023-07-08,1
4,10252,76,4,2023-07-09,2
...,...,...,...,...,...
191,10439,51,6,2024-02-07,3
192,10440,71,4,2024-02-10,2
193,10441,55,3,2024-02-10,2
194,10442,20,3,2024-02-11,2


# Задание 1: Ранжирование продуктов по средней цене
Задание: Ранжируйте продукты в каждой категории на основе их средней цены
(AvgPrice). Используйте таблицы OrderDetails и Products.
Результат: В результате запроса будут следующие столбцы:

* CategoryID: идентификатор категории продукта,
* ProductID: идентификатор продукта,
* ProductName: название продукта,
* AvgPrice: средняя цена продукта,
* ProductRank: ранг продукта внутри своей категории на основе средней цены в
порядке убывания.
Подсказка:
1. Рассчитайте среднюю цену продукта: Начните с создания подзапроса (или
CTE), в котором будете вычислять среднюю цену (AVG(Price)) для каждого
продукта. Объедините таблицы OrderDetails и Products с помощью JOIN.
2. Ранжируйте продукты по средней цене: Используйте оконную функцию
RANK() для ранжирования продуктов по средней цене внутри каждой
категории. Убедитесь, что вы применяете PARTITION BY для разделения по
категориям и ORDER BY для упорядочивания по убыванию средней цены.

Эталонное решение:
-- Рассчитываем среднюю цену для каждого продукта

* WITH ProductAvgPrice AS (
* SELECT
p.CategoryID,
p.ProductID,
p.ProductName,
AVG(p.Price) AS AvgPrice
* FROM Products p
* JOIN OrderDetails od ON p.ProductID = od.ProductID
* GROUP BY p.CategoryID, p.ProductID, p.ProductName
)

-- Ранжируем продукты по средней цене внутри каждой категории

* SELECT
CategoryID,
ProductID,
ProductName,
AvgPrice,
RANK() OVER (PARTITION BY CategoryID ORDER BY AvgPrice DESC) AS
ProductRank
* FROM ProductAvgPrice;


In [19]:
"""
with ProductAvgPrice as (select p."CategoryID", p."ProductID",p."ProductName",
AVG(p."Price") avgprice
from "Products" p 
join "OrderDetails" od on p."ProductID" = od."ProductID" 
GROUP BY p."CategoryID" , p."ProductID", p."ProductName")
SELECT *,
rank() over (partition by "CategoryID" order by "avgprice" DESC) as producrank
from ProductAvgPrice;
"""

'\nwith ProductAvgPrice as (select p."CategoryID", p."ProductID",p."ProductName",\nAVG(p."Price") avgprice\nfrom "Products" p \njoin "OrderDetails" od on p."ProductID" = od."ProductID" \nGROUP BY p."CategoryID" , p."ProductID", p."ProductName")\nSELECT *,\nrank() over (partition by "CategoryID" order by "avgprice" DESC) as producrank\nfrom ProductAvgPrice;\n'

# Задание 2: Средняя и максимальная сумма кредита по месяцам
Задание: Рассчитайте среднюю сумму кредита (AvgCreditAmount) для каждого
кластера в каждом месяце и сравните её с максимальной суммой кредита
(MaxCreditAmount) за тот же месяц. Используйте таблицу Clusters.
Подсказка:
1. Рассчитайте среднюю сумму кредита: Используйте подзапрос (или CTE) для
вычисления средней суммы кредита (AVG(credit_amount)) для каждого
кластера в каждом месяце.
2. Рассчитайте максимальную сумму кредита: Создайте другой подзапрос для
вычисления максимальной суммы кредита (MAX(credit_amount)) для каждого
месяца.
3. Объедините результаты: Используйте JOIN для объединения результатов
двух подзапросов по месяцу и выведите нужные столбцы.
Результат: В результате запроса будут следующие столбцы:
* month: месяц,
* cluster: кластер,
* AvgCreditAmount: средняя сумма кредита для каждого кластера в каждом
месяце,
* MaxCreditAmount: максимальная сумма кредита в каждом месяце.

Эталонное решение:
-- Рассчитываем среднюю сумму кредита для каждого кластера и месяца
* WITH AvgCredit AS (
SELECT
month,
cluster,
AVG(credit_amount) AS AvgCreditAmount
* FROM Clusters
* GROUP BY month, cluster
),
-- Рассчитываем максимальную сумму кредита для каждого месяца
MaxCredit AS (
* SELECT
month,
MAX(credit_amount) AS MaxCreditAmount
* FROM Clusters
* GROUP BY month
)
-- Объединяем результаты и выводим их

* SELECT
a.month,
a.cluster,
a.AvgCreditAmount,
m.MaxCreditAmount
* FROM AvgCredit a
* JOIN MaxCredit m ON a.month = m.month;

In [20]:
"""
with AvgCredit as (
select c."month" , c."cluster" ,AVG(c.credit_amount) as avgcreditamount
from "Clusters" c 
group by c."month" , c."cluster"),
MaxCredit as (select c."month" , max(c.credit_amount) as maxcreditamount
from "Clusters" c 
group by c."month")
select a."month", a."cluster", a."avgcreditamount", m."maxcreditamount"
from AvgCredit a
join MaxCredit m on a."month" = m."month"
order by a."month";
"""

'\nwith AvgCredit as (\nselect c."month" , c."cluster" ,AVG(c.credit_amount) as avgcreditamount\nfrom "Clusters" c \ngroup by c."month" , c."cluster"),\nMaxCredit as (select c."month" , max(c.credit_amount) as maxcreditamount\nfrom "Clusters" c \ngroup by c."month")\nselect a."month", a."cluster", a."avgcreditamount", m."maxcreditamount"\nfrom AvgCredit a\njoin MaxCredit m on a."month" = m."month"\norder by a."month";\n'

# Задание 3: Разница в суммах кредита по месяцам
Задание: Создайте таблицу с разницей (Difference) между суммой кредита и
предыдущей суммой кредита по месяцам для каждого кластера. Используйте таблицу
Clusters.
Подсказка:
1. Получите сумму кредита и сумму кредита в предыдущем месяце:
Используйте функцию оконного анализа LAG() для получения суммы кредита в
предыдущем месяце в рамках каждого кластера.
2. Вычислите разницу: Используйте результат предыдущего шага для
вычисления разницы между текущей и предыдущей суммой кредита.
Примените COALESCE() для обработки возможных значений NULL.
Примечания:
● Функция RANK() в MySQL 8.0 и выше позволяет вычислять ранг записей в
рамках заданного окна.
● Функция LAG() используется для получения значения предыдущей строки в
рамках окна.
● Функция MAX() может быть использована как оконная функция для получения
максимального значения в рамках окна данных.
Результат: В результате запроса будут следующие столбцы:
● month: месяц,
● cluster: кластер,
● credit_amount: сумма кредита,
● PreviousCreditAmount: сумма кредита в предыдущем месяце,
● Difference: разница между текущей и предыдущей суммой кредита.

Эталонное решение:
-- Рассчитываем сумму кредита и сумму кредита в предыдущем месяце
* WITH CreditWithPrevious AS (
* SELECT
month,
cluster,
credit_amount,
LAG(credit_amount) OVER (PARTITION BY cluster ORDER BY
month) AS PreviousCreditAmount
* FROM Clusters
)

-- Вычисляем разницу между текущей и предыдущей суммой кредита

* SELECT
month,
cluster,
credit_amount,
PreviousCreditAmount,
COALESCE(credit_amount - PreviousCreditAmount, 0) AS Difference
* FROM CreditWithPrevious;


In [21]:
"""
WITH CreditWithPrevious as (select 
c.month,
c.cluster,
c.credit_amount,
lag (c.credit_amount) over (partition by c.cluster)
as PreviousCreditAmount
from "Clusters" c )
select 
cr.month,
cr.cluster,
cr.credit_amount,
cr.PreviousCreditAmount,
COALESCE(credit_amount - PreviousCreditAmount, 0) AS Difference
from CreditWithPrevious cr;
"""

'\nWITH CreditWithPrevious as (select \nc.month,\nc.cluster,\nc.credit_amount,\nlag (c.credit_amount) over (partition by c.cluster)\nas PreviousCreditAmount\nfrom "Clusters" c )\nselect \ncr.month,\ncr.cluster,\ncr.credit_amount,\ncr.PreviousCreditAmount,\nCOALESCE(credit_amount - PreviousCreditAmount, 0) AS Difference\nfrom CreditWithPrevious cr;\n'